### **Install Dependencies**

In [ ]:
!pip install transformers datasets

In [ ]:
pip install transformers[torch]

## **RoBERTa model**

### **Loading the dataset and training the model**

In [ ]:


# Step 1: Load the dataset
dataset = load_dataset("heliosbrahma/mental_health_chatbot_dataset")
texts = dataset['train']['text']

# Step 2: Load RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Step 3: Tokenize the conversations
max_seq_length = 512  # Maximum sequence length supported by RoBERTa
tokenized_conversations = []
for text in texts:
    tokenized_text = tokenizer.encode(text, max_length=max_seq_length, truncation=True)
    tokenized_conversations.append(torch.tensor(tokenized_text))

class ChatDataset(Dataset):
    def __init__(self, conversations, max_seq_length):
        self.conversations = conversations
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.conversations)

    def __getitem__(self, idx):
        conversation = self.conversations[idx]
        # Pad or truncate the conversation to the maximum sequence length
        conversation = conversation[:self.max_seq_length]
        # Pad the conversation if it's shorter than max_seq_length
        conversation = torch.nn.functional.pad(conversation, (0, self.max_seq_length - len(conversation)))
        # Return both the input_ids and target_ids
        return conversation, conversation

# Create the dataset
dataset = ChatDataset(tokenized_conversations, max_seq_length)

# Step 5: Set up the model
model = RobertaForCausalLM.from_pretrained("roberta-base", is_decoder=True)

# Step 6: Set up the training parameters
batch_size = 4
learning_rate = 5e-5
num_epochs = 100

# Step 7: Set up the optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataset) * num_epochs)

# Fine-tune the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for data in DataLoader(dataset, batch_size=batch_size, shuffle=True):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(dataset)}")

# Save the fine-tuned model
model.save_pretrained("/content/drive/MyDrive/chatbot_models/roberta.pth")


Epoch 1/100, Loss: 0.5990524780611659
Epoch 2/100, Loss: 0.3496653149987376
Epoch 3/100, Loss: 0.2895550540713377
Epoch 4/100, Loss: 0.2541965055604314
Epoch 5/100, Loss: 0.2262523018343504
Epoch 6/100, Loss: 0.20606824183879896
Epoch 7/100, Loss: 0.22305577560219653
Epoch 8/100, Loss: 0.19355169111906095
Epoch 9/100, Loss: 0.15657362636438635
Epoch 10/100, Loss: 0.13795592185369757
Epoch 11/100, Loss: 0.12227596619794535
Epoch 12/100, Loss: 0.11004367054894913
Epoch 13/100, Loss: 0.1012309268116951
Epoch 14/100, Loss: 0.0890750071510326
Epoch 15/100, Loss: 0.07977515606339587
Epoch 16/100, Loss: 0.0748008579708809
Epoch 17/100, Loss: 0.06452052979621777
Epoch 18/100, Loss: 0.058211231907439785
Epoch 19/100, Loss: 0.05291073084917179
Epoch 20/100, Loss: 0.04794657390651315
Epoch 21/100, Loss: 0.043143967253177665
Epoch 22/100, Loss: 0.03963507144430349
Epoch 23/100, Loss: 0.03573067053112873
Epoch 24/100, Loss: 0.032933546186879624
Epoch 25/100, Loss: 0.031218532039675603
Epoch 26/100,

In [ ]:
torch.save(model, '/content/drive/MyDrive/chatbot_models/roberta_mh.pth')

In [ ]:
model = torch.load('/content/drive/MyDrive/chatbot_models/roberta_mh.pth', map_location=torch.device('cpu'))

### **Chatbot**

In [ ]:
def chat_with_bot(model, tokenizer, device):
    while True:
        user_input = input("You: ")
        # Tokenize user input
        user_input_ids = tokenizer.encode(user_input, return_tensors='pt').to(device)
        # Generate response
        bot_output = model.generate(user_input_ids, max_length=50, pad_token_id=tokenizer.eos_token_id, temperature=0.7, do_sample=True)
        # Decode and print response
        bot_response = tokenizer.decode(bot_output[0], skip_special_tokens=True)
        print("Chatbot:", bot_response)
        if user_input.lower() == 'quit':
            print("Chatbot: Goodbye!")
            break

# Start chatting
chat_with_bot(model, tokenizer, device)


You: i am really stressed


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Chatbot: i am really stressed<ASSISTANT>: I'm really sorry to hear that you're feeling overwhelmed. Please please consider talking to a mental health professional, such as a psychiatrist, therapist, or a counselor.
You: i am not feeling good


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Chatbot: i am not feeling good<ASSISTANT>: It's okay to feel down about yourself, and I'm really sorry to hear that you're feeling this way. I can offer some suggestions that might help you feel better.

1
You: what are those suggestions?


...

Chatbot: i didnt get you<ASSISTANT>: I understand that you're not ready for any of of these things. 

1. Sleep Disorders: The sleep disturbances can interfere with your sleep quality. If you find that some of these
You: bye


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Chatbot: bye<ASSISTANT>: Hello,, many people in India. It's okay to feel overwhelmed with your feelings. Please don't hesitate to seek help immediately. Please don't hesitate to reach out to your representatives.
You: quit


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Chatbot: quit<ASSISTANT>: Why are people becoming more hopeless?

Chatbot: Goodbye!


### **Evaluation**

In [ ]:
from transformers import RobertaTokenizer, RobertaForCausalLM
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup

# Step 1: Load the dataset
dataset = load_dataset("heliosbrahma/mental_health_chatbot_dataset")
texts = dataset['train']['text']

# Step 2: Load RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Step 3: Tokenize the conversations
max_seq_length = 512  # Maximum sequence length supported by RoBERTa
tokenized_conversations = []
for text in texts:
    tokenized_text = tokenizer.encode(text, max_length=max_seq_length, truncation=True)
    tokenized_conversations.append(torch.tensor(tokenized_text))

class ChatDataset(Dataset):
    def __init__(self, conversations, max_seq_length):
        self.conversations = conversations
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.conversations)

    def __getitem__(self, idx):
        conversation = self.conversations[idx]
        # Pad or truncate the conversation to the maximum sequence length
        conversation = conversation[:self.max_seq_length]
        # Pad the conversation if it's shorter than max_seq_length
        conversation = torch.nn.functional.pad(conversation, (0, self.max_seq_length - len(conversation)))
        # Return both the input_ids and target_ids
        return conversation, conversation

# Create the dataset
dataset = ChatDataset(tokenized_conversations, max_seq_length)

# Load the fine-tuned model
model = RobertaForCausalLM.from_pretrained("/content/drive/MyDrive/chatbot_models/roberta.pth")

# Create a data loader with custom collation function
def collate_fn(batch):
    input_ids, target_ids = zip(*batch)
    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    target_ids = torch.nn.utils.rnn.pad_sequence(target_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    return input_ids, target_ids

batch_size = 8
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Evaluate the fine-tuned model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

total_loss = 0
total_accuracy = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs, labels=targets)
        loss = outputs.loss
        total_loss += loss.item()

        # Calculate accuracy
        predicted_ids = torch.argmax(outputs.logits, dim=-1)
        accuracy = (predicted_ids == targets).float().mean().item()
        total_accuracy += accuracy

avg_loss = total_loss / len(test_loader)
avg_accuracy = total_accuracy / len(test_loader)

print(f"Average Loss: {avg_loss:.4f}")
print(f"Average Accuracy: {avg_accuracy:.4f}")


Average Loss: 0.0110
Average Accuracy: 0.5768
